<a href="https://colab.research.google.com/github/Msamuelsons/analise-de-dados/blob/main/tratamento_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tópicos para abordar nesse estudo de pré-processamento

In [18]:
import pandas as pd
from io import StringIO
import numpy as np

O método *StringIO* Converte uma string em valor(s), em unicode

In [3]:
csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

In [4]:
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [5]:
df.isnull()

,A,B,C,D
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


Se falta valor o retorno será True, caso contrário False

In [6]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

Uma das maneiras mais fáceis de lidar com dados ausentes é simplesmente remover totalmente os recursos
correspondentes (colunas) ou exemplos de treinamento (linhas) do conjunto de dados; linhas com valores ausentes podem
ser facilmente descartadas através do método dropna

In [7]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


axis=0, apaga as linhas com valores ausentes

axis=1, apaga a coluna com valores ausentes

In [8]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


all: Se todos os valores forem NA, elimine essa linha ou coluna.

In [11]:
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


any: Se algum valor de NA estiver presente, elimine essa linha ou coluna.



In [12]:
df.dropna(how='any')

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [13]:
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [16]:
df.dropna(thresh=4) # descarta linhas com menos de 4 valores reais

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [17]:
df.dropna(subset=['C']) #solta apenas as linhas onde NaN aparece em colunas específicas (aqui: 'C')

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


O método de drop(), nem sempre é a melhor opção pois pode acabar apagando muitos dados

# Lidando com valores faltantes

*mean imputation*


---

Uma das técnicas de interpolação mais comuns é a imputação média, onde
simplesmente substituímos o valor ausente pelo valor médio de toda a coluna de recursos.

In [23]:
from sklearn.impute import SimpleImputer
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
dados_amputados = imr.fit_transform(df.values)
dados_amputados

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

Como alternativa, uma maneira ainda mais conveniente de imputar valores ausentes é usar o método fillna dos
pandas e fornecer um método de imputação como argumento. P

In [21]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0
